In [ ]:
class TicTacToe:
    def __init__(self):
        self.N = 3
        self.map = [['E' for _ in range(self.N)] for _ in range(self.N)]    # E: 빈 공간(Empty)
        self.map_index_description = [h*self.N + w for h in range(self.N) for w in range(self.N)]
        self.player_types = ('X', 'O')  # 선공: X, 후공: O
        self.global_step = 0

        self.t_map = [[] for _ in range(7)]
        self.pos = Stack()
        self.test = [['E' for _ in range(self.N)] for _ in range(self.N)]
        self.t_global = 0; self.count = 0; self.tmp = 0; self.step = 0
        self.win = 0; self.lose = 0; self.draw = 0

        self.win_reward = 1.0
        self.defeat_reward = -1.0
        self.draw_reward = 0.0
        self.player_result = {'X': self.draw_reward, 'O': self.draw_reward}

        self.done = False

    def reset(self):
        self.map = [['E' for _ in range(self.N)] for _ in range(self.N)]
        self.global_step = 0
        self.player_result = {'X': self.draw_reward, 'O': self.draw_reward}
        self.done = False

        return self.map

    def step(self, action):
        action_coord_h, action_coord_w = self.transform_action(action)
        if self.global_step % 2 == 0:
            current_player_idx = 0
            other_player_idx = 1
        else:
            current_player_idx = 1
            other_player_idx = 0
        current_player_type = self.player_types[current_player_idx]
        other_player_type = self.player_types[other_player_idx]
        if self.map[action_coord_h][action_coord_w] == 'E':
            self.map[action_coord_h][action_coord_w] = current_player_type
            if self.is_win(current_player_type):    # 현재 플레이어 승리
                self.player_result[current_player_type] = self.win_reward
                self.player_result[other_player_type] = self.defeat_reward
                self.done = True
            elif self.is_full():    # 무승부
                self.done = True
            else:
                pass
        else:   # 현재 플레이어 패배
            self.player_result[current_player_type] = self.defeat_reward
            self.player_result[other_player_type] = self.win_reward
            self.done = True
        self.global_step += 1

        return self.map, self.player_result, self.done


    def transform_action(self, action):
        return divmod(action, self.N)

    def is_win(self, current_player_type):
        vertical_win = [True for _ in range(self.N)]
        horizontal_win = [True for _ in range(self.N)]
        diagonal_win = [True for _ in range(2)]
        for h in range(self.N):
            for w in range(self.N):
                # 가로, 세로
                if self.map[h][w] != current_player_type:
                    vertical_win[h] = False
                    horizontal_win[w] = False
                else:
                    pass
                # 왼 대각
                if h == w and self.map[h][w] != current_player_type:
                    diagonal_win[0] = False
                # 오른 대각
                rotated_w = abs(w - (self.N - 1))
                if h == rotated_w and self.map[h][w] != current_player_type:
                    diagonal_win[1] = False
        if any(vertical_win) or any(horizontal_win) or any(diagonal_win):
            return True
        else:
            return False

    def is_full(self):
        for h in range(self.N):
            for w in range(self.N):
                if self.map[h][w] == 'E':
                    return False
                else:
                    pass
        return True

    def print_description(self):
        print("** Initial NxN Tic-tac-toe Map **")
        self.print_current_map()

        print("** Action Indexes **")
        for idx, des in enumerate(self.map_index_description):
            print(des, end=' ')
            if (idx + 1) % self.N == 0:
                print('\n', end='')

    def print_current_map(self):
        for h in range(self.N):
            for w in range(self.N):
                print(self.map[h][w], end=' ')
            print('\n', end='')
        print()

    # Fill this function
    def match_prediction(self,type):
        if self.count == 0: return 
        while not len(self.pos.top[self.count]) == 0:
            if self.check(type) == 1:
               self.map = self.t_map[self.step - 2].copy
               self.step -= 1
               self.t_global -= 1
            else: 
                self.t_map[self.step] = self.map.copy
            self.find_pos()
            if self.real_move(type) != 0:
                self.step += 1
            else:
               self.map = self.t_map[self.step - 2].copy
               self.step -= 1
               self.t_global -= 1
        self.map = self.t_map[self.step].copy
        if self.win > self.draw and self.win > self.lose: print(type, "는 승리 가능성이 큽니다.")
        elif self.draw > self.win and self.draw > self.lose: print(type, "는 무승부일 가능성이 큽니다.")
        elif self.lose > self.win and self.lose > self.draw: print(type, "는 패배 가능성이 큽니다.")
        else: print("확률이 비슷하여 어느것도 확언할 수 없습니다.")


    def check(type):
        if type == 'X':
             if self.is_win('X'): self.win += 1; 
             if self.is_full():self.draw += 1; 
             if self.is_win('O'): self.lose += 1; 
        elif type == 'O': 
             if self.is_win('O'): self.win += 1; 
             elif self.is_full():self.draw += 1;
             elif self.is_win('X'): self.lose += 1;
        
        if self.lose != 0 or self.win != 0 or self.draw != 0: return 1
        else: return 0


    def find_pos(): #갈 수 있는 좌표 찾기
        for i in range(0,3):
            for j in range(0,3):
                if self.map[i][j] == 'E': self.pos.top[self.step].append((x,y))

    def real_move(type): #찾은 좌표 중 하나로 이동
        if not self.pos.top.isEmpty(self.step) == True: 
            x,y = self.pos.pop(self.step)
        else: return 0

        if type == 'X':
            if self.t_global % 2 == 0: c_type = 'X'
            else: c_type = 'O'
        elif type == 'O':
            if self.t_global % 2 == 0: c_type = 'O'
            else: c_type = 'X'

        self.map[x][y] = c_type
        self.t_global += 1
                    

class Stack: #스택을 클래스로 구현
    def __init__(self): 
        self.top = [[] for _ in range(8)]
    def isEmpty(self,i):return len(self.top[i]) == 0
    def size(self): return len(self.top)
    def clear(self): self.top = []
    def push(self,item): self.top.append(item)
    def pop(self,i):
        if not self.isEmpty(): return self.top[i].pop(-1)
    def peek(self):
        if not self.isEmpty(): return self.top[-1]


if __name__ == '__main__':
    game = TicTacToe()
    game.print_description()

    game.reset()
    done = False
    while not done:
        print()

        if game.global_step % 2 == 0:
            current_player_idx = 0
            other_player_idx = 1
        else:
            current_player_idx = 1
            other_player_idx = 0
        type = game.player_types[current_player_idx]
        game.match_prediction(type)

        action = int(input('Select action please: '))
        game.count += 1
        if not(game.map_index_description[0] <= action <= game.map_index_description[-1]):
            done = True
            print("Error: You entered the wrong number.")
            continue
        _, player_result, done = game.step(action)
        game.print_current_map()
        if done:
            for player, result in player_result.items():
                if result == game.win_reward:
                    player_result[player] = 'win'
                elif result == game.defeat_reward:
                    player_result[player] = 'defeat'
                else:
                    player_result[player] = 'draw'
            print(player_result)

######
오류가 나서 결과는 나오지 않지만, 만들려했던 알고리즘은 아래와 같습니다. 
객관적인 상황을 보고 예상하는 것이므로, 임시로 x,o를 두어 둘 수 있는 모든 경우의 수를 계산해보려 했습니다.

우선 X가 선공이므로, 맨처음 X가 둔 상태를 0단계라고 하면 단계에 맞는 순서는
1턴. X (0단계, 가장 기본 단계)
2턴. O (1단계)
3턴. X (2단계)
4턴. O (3단계)
5턴. X (4단계)
6턴. O (5단계)
7턴. X (6단계)
8턴. O (7단계)
9턴. X (8단계) 가 됩니다. 

x가 먼저 두고 나면, 그 리스트 형태(맵)을 리스트 t_map의 하나의 항목으로 넣어 임시로 저장합니다. 이 리스트에는 o,x중 한 가지가 임시로 놓아지고 단계가 올라갈 때마다 단계별 맵의 형태가 하나의 항목으로 저장됩니다. 
그리고 1단계가 되었을 때, o가 둘 수 있는 모든 좌표를 find_pos함수를 통해 찾아 스택의 한 항목에 모두 집어넣습니다. 그 후, 스택에 들어간 좌표 중 가장 마지막 것을 꺼내 그 위치에 o를 둡니다.
다시 x의 턴인 2단계가 되면, 1단계의 맵을 t_map에 저장해두고 다시 x가 둘 수 있는 모든 좌표를 찾아 스택의 다음 항목에 모두 집어넣고, 스택의 들어간 좌표 중 가장 마지막 것을 꺼내 그 위치에 x를 둡니다. 

위의 방법이 단계별로 계속해서 반복됩니다. 매단계가 반복될 때마다, 혹시 빙고가 나왔는지를 확인합니다. 8단계까지 가지 않아도 빙고가 나왔을 경우 그 다음단계까지 갈 필요가 없음으로, 현재 단계의 스택을 모두 비우고 이전 단계의 스택으로 되돌아가 그 스택의 남은 좌표를 찾아냅니다(step변수가 단계를 나타냅니다). 맵의 경우, 단계를 두 번 앞으로 가야 이전 단계의 수를 둘 수 있으므로 t_map에 저장해두었던 단계별 맵의 형태 중 두 단계전의 형태를 map으로 불러옵니다. 

만약 8단계까지 도달하게 되었을 때도, 9칸이 모두 찾다면 어찌되었든 졌거나, 이겼거나, 비겼다는 결과가 나오므로 알고리즘은 빙고가 나왔을 때와 같습니다. 스택은 현재 스택을 모두 비운 후, 한 단계 전으로, 맵은 두 단계 전으로 되돌아갑니다. 

또한, 현재 단계의 스택이 모두 비었을 경우(real_move함수에서 0이 반환됩니다), 빙고와 나왔을 때와 마찬가지로 맵을 두 단계전으로, 스택을 한 단계 전으로 돌아갑니다. 다만 이때는 스택에 이미 항목이 없으므로 스택을 비울 필요없습니다.

그렇게 가장 깊은 단계로 먼저 파고들었다가 이전 상태로 돌려 다시 파고 들고, 다시 이전 단계로 돌아와 다시 파고 드는 과정을 거치다 제일 처음의 단계로 돌아왔을 때, 모든 경우의 수를 확인하면서 세어두었던 win, lose, draw 값들을 비교해 가장 큰 값을 가진 변수가 최대 예측이 되는 것입니다.

***
위까지가 구현하려고 했던 알고리즘의 내용입니다. 그런데 정확한 원인은 못 찾았으나 함수들을 모두 만들고 나니 메인함수의 game.step함수를 부르는 과정에서 int값을 불러오지 못한다는 오류가 발생합니다. 제 예상으로는 변수나 리스트들을 만드는 과정에서 뭔가 충돌이 생긴 것 같은데, 정확한 이유는 찾지 못했습니다...

